In [ ]:
%matplotlib inline

Preparations
------------


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import pandas as pd



USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

#Data cleaning

In [ ]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 32.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394489 sha256=30e51dc0b59b001d3ce798a89b4a3e3a72ec12c82821346e406ac5b485fcfb3a
  Stored in directory: /root/.cache/pip/wheels/6b/6d/14/3defa4cd7013faeddf715150696f4a96d7725c87700eb8a68e
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp310-cp310-linux_x86_64.whl size=180376 sha256=27184d9f08f1ba688047a6d5fb5b450e0052babea086908259d5960335ee3635
  Stored in directory: /root/.cache/pip/wheels/9f/cb/30/fef48ecac051e433987eccdb5682900b4c00d44a4bcd4d4ec8
Successfully built nltk libwapiti
  Attempting un

In [ ]:
import numpy as np
import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from __future__ import unicode_literals
from hazm import *
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<div dir='rtl'>
حذف حروف انگلیسی

حذف ایموجی ها

حذف علایم نگارشی تکرار شده مثل ....

حذف حروف تکرار شونده بیشتر از سه بار

نرمالایز کردن متن


In [ ]:
normalizer = Normalizer()
def process(full_text):
    cleaned_text = clean_tweet(text=full_text)
    normalized_text = normalizer.normalize(cleaned_text)
    normalized_text = re.sub('\u200c',' ',normalized_text)#???
    tokenized_text = tokenizer_hazm(normalized_text)
    return " ".join(tokenized_text)

def clean_tweet(text=None):
    """ get text of tweet and clean it.
        cleaning is done for:
        1- urls
        2- emoticons
        3- mentions from the beginning of text
        4- remove hashtags from the beginning and end of text
        
        5- delete english charachters
        6- delete repeated punctuations 
        7- remove repeated chatacter that repeate more than 3 times

    :param text: fulltext of tweet
    :return: cleaned text
    """

    full_text = text

    # remove urls
    regex = re.compile('https:\S*')
    it = re.finditer(regex, full_text)
    for match in it:
        full_text = full_text.replace(match.group(0), "")

    # remove emoticons
    emoji_pattern = re.compile(u'['
                               u'\U0001F300-\U0001F64F'
                               u'\U0001F680-\U0001F6FF'
                               u'\u2600-\u26FF\u2700-\u27BF]+',
                               re.UNICODE)
    full_text = emoji_pattern.sub('', full_text)

    # remove mentions
    regex = re.compile('@\w*')
    it = re.finditer(regex, full_text)
    for match in it:
        full_text = full_text.replace(match.group(0), "")

    # remove mention sign from rest of text
    full_text = full_text.replace("@", "")

    # remove hashtags at the end of string
    regex = re.compile('([\n\s]*#\w*)*\Z')
    it = re.finditer(regex, full_text)
    for match in it:
        full_text = full_text.replace(match.group(0), "")

    # remove hashtags in the begining of string
    regex = re.compile('\A([\s\n]*#\w*\s*\n+\s*)*')
    it = re.finditer(regex, full_text)
    for match in it:
        full_text = full_text.replace(match.group(0), "")

    full_text = full_text.replace("#", "")
    full_text = full_text.replace("RT", "")

    # delete english charachters
    full_text = re.sub('[^\u0600-\u06FF\s]','',full_text)

    # delete repeated punctuations
    rx = re.compile(r'([!-/:-@[-`{-~])\1+')
    full_text = rx.sub(r'\1 ', full_text)

    #convert half space to space
    #? hazm added ?
    full_text =  re.sub('\u200c',' ',full_text)
    # remove repeated chatacter that repeate more than 3 times
    #?

    return full_text.strip()


def tokenizer_hazm(text):
    tokenized = []
    for sent in sent_tokenize(text):
        tokenized.extend(word_tokenize(sent))
    return tokenized

#Format Dataset

##PerCQA


In [ ]:
df_p = pd.read_csv('PerCQA_CSV_Format.csv',error_bad_lines = False)

<ipython-input-29-31cd85f38b6c>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_p = pd.read_csv('PerCQA_CSV_Format.csv',error_bad_lines = False)
Skipping line 652: expected 10 fields, saw 12
Skipping line 1231: expected 10 fields, saw 11
Skipping line 1747: expected 10 fields, saw 11
Skipping line 1781: expected 10 fields, saw 19
Skipping line 2519: expected 10 fields, saw 12
Skipping line 2523: expected 10 fields, saw 11
Skipping line 2747: expected 10 fields, saw 11
Skipping line 2953: expected 10 fields, saw 11
Skipping line 2958: expected 10 fields, saw 12
Skipping line 2959: expected 10 fields, saw 11
Skipping line 3189: expected 10 fields, saw 11
Skipping line 3204: expected 10 fields, saw 14
Skipping line 3206: expected 10 fields, saw 11
Skipping line 3242: expected 10 fields, saw 12
Skipping line 3554: expected 10 fields, saw 11
Skipping line 3674: expected 10 fields, saw 11
Skipp

In [ ]:
len(df_p)

21999

In [ ]:
df_p.head()

,QID,QUsername,QDATE,QSubject,QBody,CID,CUSERID,CUsername,CBodyClean,CGOLD
0,3270053,سوشااا,2019/09/21 12:50,قد دختر سه ساله,دخترم۳سالو نیمه قدش ۹۵ وزنش۱۴ خوبه یا کمه,105626741,32122,خوششانسم,پسرم یه ماه دیگه میشه سه سال.قدش حدودا دو ماه ...,Good
1,3270053,سوشااا,2019/09/21 12:50,قد دختر سه ساله,دخترم۳سالو نیمه قدش ۹۵ وزنش۱۴ خوبه یا کمه,105626772,1884,مربا,نمیدونم پسرم 25 ماهشه قدش 95. خودتون قدتون چجو...,Good
2,3270053,سوشااا,2019/09/21 12:50,قد دختر سه ساله,دخترم۳سالو نیمه قدش ۹۵ وزنش۱۴ خوبه یا کمه,105626911,39292,asalbanojonam,من پسرم دوسال ونه ماهش هس قد93وزن 13ترخدا بگید...,Good
3,3270053,سوشااا,2019/09/21 12:50,قد دختر سه ساله,دخترم۳سالو نیمه قدش ۹۵ وزنش۱۴ خوبه یا کمه,105626962,33018,متولدماه7,عزیزم بچه هارو مقایسه نکن .حتی خواهر بردار ها ...,Good
4,3270053,سوشااا,2019/09/21 12:50,قد دختر سه ساله,دخترم۳سالو نیمه قدش ۹۵ وزنش۱۴ خوبه یا کمه,105627088,3449,Miss.iDa,دختر منم 3 سالشه قدش یه ماه پیش 89 بود وزنش سی...,Good


In [ ]:
df_p = df_p [['QBody','CBodyClean']]

In [ ]:
np.savetxt(r'PerCQA_pre_formatted.txt', df_p.values, fmt='%s', delimiter='\t')

In [ ]:
lines_p = open('PerCQA_pre_formatted.txt', encoding='utf-8').read().strip().split('\n')
pairs_p = [[process(s) for s in l.split('\t')] for l in lines_p]

In [ ]:
df1_p = pd.DataFrame(pairs_p)

In [ ]:
df1_p.head()

,0,1
0,دخترم۳سالو نیمه قدش ۹۵ وزنش ۱۴ خوبه یا کمه,پسرم یه ماه دیگه میشه سه سالقدش حدودا دو ماه پ...
1,دخترم۳سالو نیمه قدش ۹۵ وزنش ۱۴ خوبه یا کمه,نمیدونم پسرم ماهشه قدش خودتون قدتون چجوریه ؟
2,دخترم۳سالو نیمه قدش ۹۵ وزنش ۱۴ خوبه یا کمه,من پسرم دوسال ونه ماهش هس قدوزن ترخدا بگید خوب...
3,دخترم۳سالو نیمه قدش ۹۵ وزنش ۱۴ خوبه یا کمه,عزیزم بچه هارو مقایسه نکن حتی خواهر بردار ها ه...
4,دخترم۳سالو نیمه قدش ۹۵ وزنش ۱۴ خوبه یا کمه,دختر منم سالشه قدش یه ماه پیش بود وزنش سیزده و...


In [ ]:
df1_p.to_csv('PerCQA_clean.csv')

In [ ]:
np.savetxt(r'PerCQA_formatted.txt', df1_p.values, fmt='%s', delimiter='\t')

In [ ]:
# emoji_pattern = re.compile("["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            "]+", flags=re.UNICODE)

In [ ]:
# rx = re.compile(r'([!-/:-@[-`{-~])\1+')
# texto = rx.sub(r'\1 ', 'sss..???. ... --- !!!!')
# print(re. sub('\s{2,}', ' ', texto))
# print(len(texto))
# print(texto)

In [ ]:
# 'g  gg '.strip()

In [ ]:
# re.sub(r'\s{2,}', r'', 'ممممکن نیست')

In [ ]:
# re.sub(r'([\u0600-\u06FF\s]){2,}', r'\1', 'ممممکن نیست')

In [ ]:
# def square(match):
#     num = (match.group())
#     print(match.group(1))
#     return str(num)

# l = ['ممممکن']
# pattern = r'([\u0600-\u06FF\s]){2,}'
# new_l = [re.sub(pattern, square, s) for s in l]


In [ ]:
# new_l

In [ ]:
# re.sub(r'([\u0600-\u06FF\s])\{2,}', r'\1', 'ممممکن')

In [ ]:
# rx.sub(r'\1 ', 'dada?//???? dd??')

In [ ]:
# df1 = pd.DataFrame()
# df1['QBody'] = df_s['QBody'].apply(lambda row: re.sub('[^\u0600-\u06FF\s]',"",normalizer.normalize(emoji_pattern.sub(r'', str(row).replace('\n',' ')))))
# df1['CBodyClean'] = df_s['CBodyClean'].apply(lambda row: re.sub('[^\u0600-\u06FF\s]',"",normalizer.normalize(emoji_pattern.sub(r'', str(row).replace('\n',' ')))))

In [ ]:
# rx = re.compile(r'([!-/:-@[-`{-~])\1+')

# df1['QBody'] = df1['QBody'].apply(lambda row: rx.sub(r'\1 ', str(row)))
# df1['CBodyClean'] = df1['CBodyClean'].apply(lambda row: rx.sub(r'\1 ', str(row)))

# df1['QBody'] = df1['QBody'].apply(lambda row: re.sub('\s{2,}', ' ', str(row)).strip())
# df1['CBodyClean'] = df1['CBodyClean'].apply(lambda row: re.sub('\s{2,}', ' ', str(row)).strip())


In [ ]:
# df1.head(10)

##ChitChat dataset

In [ ]:
df_ch = pd.read_csv('ChatbotData.csv',error_bad_lines = False)

<ipython-input-37-83937a3ab35b>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_ch = pd.read_csv('ChatbotData.csv',error_bad_lines = False)


In [ ]:
df_ch.head()

,Q,A
0,در ھر صورت خیلی ممنون.,خواھش میکنم.
1,سلام خوبین؟,سلام ممنونم.
2,سلام کجایی؟,سلام خونم.
3,اوکی مرسی,خواھش
4,مرسی کاکو,فدات مخلصیم


In [ ]:
np.savetxt(r'Chitchat_pre_formatted.txt', df_ch.values, fmt='%s', delimiter='\t')

In [ ]:
lines_ch = open('Chitchat_pre_formatted.txt', encoding='utf-8').read().strip().split('\n')
pairs_ch = [[process(s) for s in l.split('\t')] for l in lines_ch]

In [ ]:
pairs_ch[0]

['در ھر صورت خیلی ممنون', 'خواھش میکنم']

In [ ]:
df1_ch = pd.DataFrame(pairs_ch)

In [ ]:
df1_ch.to_csv('chitchat_clean.csv')

In [ ]:
df1_ch.head()

,0,1
0,در ھر صورت خیلی ممنون,خواھش میکنم
1,سلام خوبین ؟,سلام ممنونم
2,سلام کجایی ؟,سلام خونم
3,اوکی مرسی,خواھش
4,مرسی کاکو,فدات مخلصیم


In [ ]:
np.savetxt(r'Chitchat_formatted.txt', df1_ch.values, fmt='%s', delimiter='\t')

##Combination of PerCQA and chitchat

In [ ]:
df1_comb = pd.concat([df1_ch,df1_p])

In [ ]:
df1_comb.iloc[2023][0]

'به بچه های دو سالتون چی یاد میدین ؟ من نمیدونم دیگه باید چی یادش بدم ؟'

In [ ]:
np.savetxt(r'Combination_formatted.txt', df1_comb.values, fmt='%s', delimiter='\t')

In [ ]:
df1_comb.to_csv('chitchat_percqa_clean.csv')

#Dataset Configs

In [ ]:
config = {
    'PerCQA' : {
        'corpus_name' : "ninsite", 
        'datafile' : "PerCQA_formatted.txt",
        'MAX_LENGTH' : 80,
        'MIN_COUNT' : 3,
        'model_name' : 'p_model',
        'attn_model' : 'dot',
        'hidden_size' : 500,
        'encoder_n_layers' : 2,
        'decoder_n_layers' : 2,
        'dropout' : 0.1,
        'batch_size' : 64,
        'train' : {
            'clip' : 50.0,
            'teacher_forcing_ratio' : 1.0,
            'learning_rate' : 0.0001,
            'decoder_learning_ratio' : 5.0,
            'n_iteration' : 3000,
            'print_every' : 100,
            'save_every' : 100

        }



    },
    'Chitchat' : {
        'corpus_name' : "chitchat", 
        'datafile' : "Chitchat_formatted.txt",
        'MAX_LENGTH' : 10,
        'MIN_COUNT' : 2,
        'model_name' : 'ch_model',
        'attn_model' : 'dot',
        'hidden_size' : 500,
        'encoder_n_layers' : 2,
        'decoder_n_layers' : 2,
        'dropout' : 0.1,
        'batch_size' : 64,
        'train' : {
            'clip' : 50.0,
            'teacher_forcing_ratio' : 1.0,
            'learning_rate' : 0.0001,
            'decoder_learning_ratio' : 5.0,
            'n_iteration' : 400,
            'print_every' : 20,
            'save_every' : 50

        }

    },
    'PerCQA_Chitchat' : {
        'corpus_name' : "comb", 
        'datafile' : "Combination_formatted.txt",
        'MAX_LENGTH' : 30,
        'MIN_COUNT' : 2,
        'model_name' : 'cb_model',
        'attn_model' : 'dot',
        'hidden_size' : 500,
        'encoder_n_layers' : 2,
        'decoder_n_layers' : 2,
        'dropout' : 0.1,
        'batch_size' : 64,
        'train' : {
            'clip' : 50.0,
            'teacher_forcing_ratio' : 1.0,
            'learning_rate' : 0.0001,
            'decoder_learning_ratio' : 5.0,
            'n_iteration' : 4000,
            'print_every' : 10,
            'save_every' : 50

        }

    }
}

Load & Preprocess Data
----------------------


In [ ]:
#Dataset that use for training
DATASET = 'PerCQA_Chitchat'
CONFIG = config[DATASET]
TRAIN_CONFIG = CONFIG['train']

In [ ]:
def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

# Load and trim data
~~~~~~~~~~~~~~~~~~


In [ ]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token
UNK_token = 3  # Unknown token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", 
                           UNK_token: "UNK" }
        self.num_words = 4  # Count SOS, EOS, PAD, UNK

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", 
                           UNK_token: "UNK" }
        self.num_words = 4  # Count SOS, EOS, PAD, UNK

        for word in keep_words:
            self.addWord(word)

In [ ]:
MAX_LENGTH = CONFIG['MAX_LENGTH'] # Maximum sentence length to consider


# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[s for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    if len(p) <= 1:
        return False
    if len(p[0]) == 0 or len(p[1]) == 0:
        return False
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
corpus_name = CONFIG['corpus_name']
datafile = CONFIG['datafile']

voc, pairs = loadPrepareData ( corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 22850 sentence pairs
Trimmed to 9579 sentence pairs
Counting words...
Counted words: 14041

pairs:
['در ھر صورت خیلی ممنون', 'خواھش میکنم']
['سلام خوبین ؟', 'سلام ممنونم']
['سلام کجایی ؟', 'سلام خونم']
['اوکی مرسی', 'خواھش']
['مرسی کاکو', 'فدات مخلصیم']
['سلام خوبین ؟', 'سلام ممنون شما خوبین ؟']
['ی سوال دارم', 'بفرمایید']
['خیلی ممنونم ، برم بررسیشون کنم', 'موفق باشید']
['سلام میشه بپرسم تمرینتون چیه ؟', 'سلام بله اجازه بدید']
['ببخشید نمیرسم انجامش بدم', 'خواهش میکنم مشکلی نیست']


In [ ]:
MIN_COUNT = CONFIG['MIN_COUNT']   # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 6466 / 14037 = 0.4606
Trimmed from 9579 pairs to 5224, 0.5454 of total


In [ ]:
len(pairs)

5224

In [ ]:
voc.word2index['سلام']


11

In [ ]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[1459,   77, 3449,  384,  384],
        [ 243, 1406, 3023, 2627,  208],
        [  84,  136,  499, 2995,  147],
        [2053, 1692, 5977, 2996, 5328],
        [1520,  731,  602,  243,  330],
        [1649,   13, 1415,  338, 1321],
        [  77, 1185, 4800,  313, 5329],
        [  78, 1692,  624, 5128,  269],
        [ 269,  821,   13,    8, 3260],
        [ 103,  179, 5978,  444,  210],
        [2054,   60,  133, 1324,   13],
        [2055,   13,  147, 2334,    2],
        [  56, 6287, 5979,  465,    0],
        [1520,   13, 3441,    2,    0],
        [ 989,  512,   13,    0,    0],
        [ 998,   30,    2,    0,    0],
        [  50,  488,    0,    0,    0],
        [1785, 1644,    0,    0,    0],
        [ 621,   13,    0,    0,    0],
        [2056,    2,    0,    0,    0],
        [ 208,    0,    0,    0,    0],
        [2057,    0,    0,    0,    0],
        [2058,    0,    0,    0,    0],
        [ 242,    0,    0,    0,    0],
        [ 512,    0,    

In [ ]:
[random.choice(pairs) for _ in range(small_batch_size)]

[['دادن سیپرو برای اشتهای بچه ضرر داره ؟ اصلا اشتها نداره ماهشه کلی شربت اشتها هم دادم ولی فایده نداشته',
  'چقدر میدی چجوری ؟ بچه چند وقتشه ؟'],
 ['شیرخشک پپتی جونیوره نمیخوره گفتم شاید مزش عوض بشه بخوره',
  'مگه خودت میخای بخوری ؟'],
 ['شما چه قطره آهنی برای کوچولوهاتون استفاده می کنین ؟؟ راضی هستین می خوره بالا نمیاره و دندونشو سیاه نمی کنه ؟ قیمتش رو هم بگین ممنون',
  'تو داروخانه بری بگی راهنماییم کنین بهتر از اینجا راهنمایی میکنن چون بلد هستن'],
 ['بهترین جنس قابلمه چیه برای جهیزیه میخوام اصلا قیمتش مهم نیست میخوام از هر لحاظ عالی ترین باشه',
  'گرانیت'],
 ['بچهه ها میدونین مهمون امشب دور همى کیه ؟؟',
  'محمود کلاری حسش نبود خاموشیدم']]

# Define Models



## Encoder

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0, block_type = 'gru'):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.block_type = block_type
        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        if block_type == 'gru' :
            self.block = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)
        else:
            self.block = nn.LSTM(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

        
    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through Block
        if self.block_type == 'gru':
            outputs, hidden = self.block(packed, hidden)
        else:
            #inthis case hodden is a tuple that contains both last hidden state and last cell state
            outputs, hidden = self.block(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional Block outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

## Decoder


In [ ]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1, block_type='gru'):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.block_type = block_type
        
        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        if block_type == 'gru' :
            self.block = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        
        else:
            self.block = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # print('there')
        # print(embedded.shape)
        # print(last_hidden[0].shape)

        # Forward through unidirectional block
        if self.block_type == 'gru':
            rnn_output, hidden = self.block(embedded, last_hidden)
        
        else:
            rnn_output, (hidden) = self.block(embedded, last_hidden)
        
        
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

# Define Training Procedure



## Masked loss

In [ ]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [ ]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [ ]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

Define Evaluation
-----------------



In [ ]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        print(type(input_seq))
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [ ]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    # lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = process(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

Run Model
---------


In [ ]:
# Configure models
model_name = CONFIG['model_name']
attn_model = CONFIG['attn_model']
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = CONFIG['hidden_size']
encoder_n_layers = CONFIG['encoder_n_layers']
decoder_n_layers = CONFIG['decoder_n_layers']
dropout = CONFIG['dropout']
batch_size = CONFIG['batch_size']

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [ ]:
# Configure training/optimization
clip = TRAIN_CONFIG['clip']
teacher_forcing_ratio = TRAIN_CONFIG['teacher_forcing_ratio']
learning_rate = TRAIN_CONFIG['learning_rate']
decoder_learning_ratio = TRAIN_CONFIG['decoder_learning_ratio']
n_iteration = TRAIN_CONFIG['n_iteration']
print_every = TRAIN_CONFIG['print_every']
save_every = TRAIN_CONFIG['save_every']

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
    
# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 10; Percent complete: 0.2%; Average loss: 8.4109
Iteration: 20; Percent complete: 0.5%; Average loss: 7.5192
Iteration: 30; Percent complete: 0.8%; Average loss: 6.8372
Iteration: 40; Percent complete: 1.0%; Average loss: 6.8903
Iteration: 50; Percent complete: 1.2%; Average loss: 6.8070
Iteration: 60; Percent complete: 1.5%; Average loss: 6.7663
Iteration: 70; Percent complete: 1.8%; Average loss: 6.7061
Iteration: 80; Percent complete: 2.0%; Average loss: 6.6840
Iteration: 90; Percent complete: 2.2%; Average loss: 6.6432
Iteration: 100; Percent complete: 2.5%; Average loss: 6.5869
Iteration: 110; Percent complete: 2.8%; Average loss: 6.5344
Iteration: 120; Percent complete: 3.0%; Average loss: 6.5193
Iteration: 130; Percent complete: 3.2%; Average loss: 6.5153
Iteration: 140; Percent complete: 3.5%; Average loss: 6.4941
Iteration: 150; Percent complete: 3.8%; Average loss: 6.4760
Iteration: 160; Percen

In [ ]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

> امروز چیکار کردی ؟
<class 'torch.Tensor'>
Bot: هیچی همش خواب بودم و فیلم دیدم که خیالت راحت بشه یا حداقل داره ولی تو جمع میشه کار میکنه تو زندگی شده
> خدا حافظ
Error: Encountered unknown word.
> خداحافظ
Error: Encountered unknown word.
> خدافظ
<class 'torch.Tensor'>
Bot: به سلامت کاکو که میتونی تو لیاقت خوب بشه که میشی زود که بهت باشم تا ساعت اثر داره که از سالگی سالی زندگی شد یه
> بهش چی بگم ؟
<class 'torch.Tensor'>
Bot: هیچی ندادم تا سال دیگه آهن جدا که میتونی بره چون خونه تا دختر بود نه که خیالت راحت بشه میتونی ازش بگیر


KeyboardInterrupt: ignored

In [ ]:
pairs[1101]

['فردا واکسن ۱۸ ماهگی پسرمه چه کارهای بکنم زود خوب بشه از استرسش خوابم نمیاد',
 'قبل رفتن استامینوفن بده کمپرس هم طبق چیزی ک میگن و اینکه حتی تب نداره استامینوفن بده ک درد اذیتش نکنه']